In [1]:

from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [2]:
import csv


f = open("lekmer.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("lekmer.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,url,img_url,article_no,blob_url


In [2]:
categorys=['accessories','badklader and uvklader','bodies and sparkräkter',
          'Dopkläder & högtidskläder','klanningar','nederlander',
          'regnklader','ryggsackor and vaskor','sovklader','sportklader',
          'underklader','tecknat serier','yttklader','fleeceklader',
          'overdelear','jul and luciaklader','shoes']
sub_urls=['https://lekmer.se/barnklader-accessoarer/s/2006',
          'https://lekmer.se/barnklader-badklader-uv-klader/s/1973',
         'https://lekmer.se/barnklader-bodies-sparkdrakter/s/1929',
         'https://lekmer.se/barnklader-dopklader-hogtidsklader/s/1984',
         'https://lekmer.se/barnklader-klanningar/s/1933',
          'https://lekmer.se/barnklader-nederdelar/s/1920',
         'https://lekmer.se/barnklader-regnklader/s/1965',
         'https://lekmer.se/barnklader-ryggsackar-vaskor/s/1995',
         'https://lekmer.se/barnklader-sovklader/s/1981',
         'https://lekmer.se/barnklader-sport-drottsklader/s/1943',
         'https://lekmer.se/barnklader-underklader-strumpor/s/1936',
         'https://lekmer.se/tecknat-serier/s/2642',
         'https://lekmer.se/barnklader-ytterklader/s/1946',
         'https://lekmer.se/barnklader-ytterklader-fleeceklader-forstarkningsplagg/s/1958',
         'https://lekmer.se/barnklader-overdelar/s/1910',
         'https://lekmer.se/barnklader-jul-uciaklader/s/1991',
         'https://lekmer.se/barnskor/s/2014']

In [3]:
for c,i in enumerate(range(13)):
    categorys.pop(0)
    sub_urls.pop(0)

In [4]:
categorys


['fleeceklader', 'overdelear', 'jul and luciaklader', 'shoes']

In [4]:
!mkdir lekmer

mkdir: cannot create directory ‘lekmer’: File exists


In [5]:
import json
import requests
import time
from random import randint
from bs4 import BeautifulSoup
import urllib.request
import csv
gender='unisex'
for count,sub_url in enumerate(sub_urls):
    category=categorys[count]
    html_doc=requests.get(sub_url).content.decode()
    soup = BeautifulSoup(html_doc, 'html.parser')
    urls=[sub_url]
    try:
        while True:
            time.sleep(3)
            pages=soup.find("div",attrs={"class":"pagination-buttons"})
            page_url='https://lekmer.se'+pages.find("a",attrs={"class":"next"})['href']
            urls.append(page_url)
            html_doc=requests.get(page_url).content.decode()
            soup = BeautifulSoup(html_doc, 'html.parser')
    except:
        pass
    for page in urls:
        html_doc2=requests.get(page).content.decode()
        soup2 = BeautifulSoup(html_doc2, 'html.parser')
        products=soup2.find_all("div",attrs={"class":"card-image-container"})
        print(page)
        print(len(products))
        
        for p in products:
            url='https://lekmer.se'+p.a['href']
            html_doc3=requests.get(url).content.decode()
            soup3 = BeautifulSoup(html_doc3, 'html.parser')
            brand=soup3.find("a",attrs={"class":"card-brand"}).text
            name=soup3.find("h1",attrs={"class":"product-title"}).span.text
            price=soup3.find("div",attrs={"class":"product-price"}).select('[data-price]')[0].text.strip()

            img_url='https://lekmer.se'+soup3.find("div",attrs={"class":"product-img-container"}).find("img")['data-href']
            try:
                article_no=soup3.find("span",attrs={"class":"property-box-value"}).text
            except:
                continue
            
            ac='lekmer'+'/'+article_no+str('.jpg')
#             opener = urllib.request.URLopener()
#             opener.addheader('User-Agent', 'whatever')
#             filename, headers2 = opener.retrieve(img_url, ac)

            rr = requests.get(img_url, allow_redirects=True)
            open(ac, 'wb').write(rr.content)

            blob_name=str('lekmer')+str('/')+str(article_no)
            path_to_file='lekmer/'+article_no+str('.jpg')
            blob_url=upload_to_bucket(blob_name, path_to_file)

            sizes=soup3.find_all("div",attrs={"class":"dropdown-menu"})
            for sz in sizes:
                size=sz.a.text
                with open("lekmer.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,url,img_url,article_no,blob_url])
        

https://lekmer.se/barnklader-accessoarer/s/2006
40
https://lekmer.se/barnklader-accessoarer/s/2006/2
40
https://lekmer.se/barnklader-accessoarer/s/2006/3
40
https://lekmer.se/barnklader-accessoarer/s/2006/4
40
https://lekmer.se/barnklader-accessoarer/s/2006/5
40
https://lekmer.se/barnklader-accessoarer/s/2006/6
40
https://lekmer.se/barnklader-accessoarer/s/2006/7
40
https://lekmer.se/barnklader-accessoarer/s/2006/8
40
https://lekmer.se/barnklader-accessoarer/s/2006/9
40
https://lekmer.se/barnklader-accessoarer/s/2006/10
40
https://lekmer.se/barnklader-accessoarer/s/2006/11
40
https://lekmer.se/barnklader-accessoarer/s/2006/12
40
https://lekmer.se/barnklader-accessoarer/s/2006/13
40
https://lekmer.se/barnklader-accessoarer/s/2006/14
40
https://lekmer.se/barnklader-accessoarer/s/2006/15
40
https://lekmer.se/barnklader-accessoarer/s/2006/16
40
https://lekmer.se/barnklader-accessoarer/s/2006/17
40
https://lekmer.se/barnklader-accessoarer/s/2006/18
40
https://lekmer.se/barnklader-accessoare

https://lekmer.se/barnklader-sport-drottsklader/s/1943/5
40
https://lekmer.se/barnklader-sport-drottsklader/s/1943/6
37
https://lekmer.se/barnklader-underklader-strumpor/s/1936
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/2
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/3
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/4
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/5
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/6
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/7
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/8
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/9
40
https://lekmer.se/barnklader-underklader-strumpor/s/1936/10
21
https://lekmer.se/tecknat-serier/s/2642
40
https://lekmer.se/tecknat-serier/s/2642/2
40
https://lekmer.se/tecknat-serier/s/2642/3
40
https://lekmer.se/tecknat-serier/s/2642/4
40
https://lekmer.se/tecknat-serier/s/2642/5
40
https://lekmer.se/tecknat-serier/s/264

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
url

In [ ]:
soup3.find("div",attrs={"class":"product-img-container"}).find("img")['data-href']

In [5]:
import json
import requests
import time
from random import randint
from bs4 import BeautifulSoup
import urllib.request
import csv
gender='unisex'
for count,sub_url in enumerate(sub_urls):
    category=categorys[count]
    html_doc=requests.get(sub_url).content.decode()
    soup = BeautifulSoup(html_doc, 'html.parser')
    urls=[sub_url]
    try:
        while True:
            time.sleep(3)
            pages=soup.find("div",attrs={"class":"pagination-buttons"})
            page_url='https://lekmer.se'+pages.find("a",attrs={"class":"next"})['href']
            urls.append(page_url)
            html_doc=requests.get(page_url).content.decode()
            soup = BeautifulSoup(html_doc, 'html.parser')
    except:
        pass
    for page in urls:
        html_doc2=requests.get(page).content.decode()
        soup2 = BeautifulSoup(html_doc2, 'html.parser')
        products=soup2.find_all("div",attrs={"class":"card-image-container"})
        print(page)
        print(len(products))
        
        for p in products:
            url='https://lekmer.se'+p.a['href']
            html_doc3=requests.get(url).content.decode()
            soup3 = BeautifulSoup(html_doc3, 'html.parser')
            brand=soup3.find("a",attrs={"class":"card-brand"}).text
            name=soup3.find("h1",attrs={"class":"product-title"}).span.text
            price=soup3.find("div",attrs={"class":"product-price"}).select('[data-price]')[0].text.strip()

            img_url='https://lekmer.se'+soup3.find("div",attrs={"class":"product-img-container"}).find("img")['data-href']
            try:
                article_no=soup3.find("span",attrs={"class":"property-box-value"}).text
            except:
                continue
            
            ac='lekmer'+'/'+article_no+str('.jpg')
#             opener = urllib.request.URLopener()
#             opener.addheader('User-Agent', 'whatever')
#             filename, headers2 = opener.retrieve(img_url, ac)

            rr = requests.get(img_url, allow_redirects=True)
            open(ac, 'wb').write(rr.content)

            blob_name=str('lekmer')+str('/')+str(article_no)
            path_to_file='lekmer/'+article_no+str('.jpg')
            blob_url=upload_to_bucket(blob_name, path_to_file)

            sizes=soup3.find_all("div",attrs={"class":"dropdown-menu"})
            for sz in sizes:
                size=sz.a.text
                with open("lekmer.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,url,img_url,article_no,blob_url])
        

https://lekmer.se/barnklader-ytterklader-fleeceklader-forstarkningsplagg/s/1958
40
https://lekmer.se/barnklader-ytterklader-fleeceklader-forstarkningsplagg/s/1958/2
40
https://lekmer.se/barnklader-ytterklader-fleeceklader-forstarkningsplagg/s/1958/3
40
https://lekmer.se/barnklader-ytterklader-fleeceklader-forstarkningsplagg/s/1958/4
40
https://lekmer.se/barnklader-ytterklader-fleeceklader-forstarkningsplagg/s/1958/5
11
https://lekmer.se/barnklader-overdelar/s/1910
40
https://lekmer.se/barnklader-overdelar/s/1910/2
40
https://lekmer.se/barnklader-overdelar/s/1910/3
40
https://lekmer.se/barnklader-overdelar/s/1910/4
40
https://lekmer.se/barnklader-overdelar/s/1910/5
40
https://lekmer.se/barnklader-overdelar/s/1910/6
40
https://lekmer.se/barnklader-overdelar/s/1910/7
40
https://lekmer.se/barnklader-overdelar/s/1910/8
40
https://lekmer.se/barnklader-overdelar/s/1910/9
40
https://lekmer.se/barnklader-overdelar/s/1910/10
40
https://lekmer.se/barnklader-overdelar/s/1910/11
40
https://lekmer.s

In [6]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json('auth.json')
bucket = storage_client.get_bucket('urbankiddo')
blob = bucket.blob('lekmer.csv')
blob.upload_from_filename('lekmer.csv')
blob.public_url

'https://storage.googleapis.com/urbankiddo/lekmer.csv'

In [ ]:
soup3.find("div",attrs={"class":"gallery-carousel-items"}).find("img")['data-href']
            

In [ ]:
soup3.find("div",attrs={"class":"gallery-carousel-items"})
            

In [ ]:
p_price=soup3.find("div",attrs={"class":"product-price"}).select('[data-price]')[0].text.strip()
p_price

In [ ]:
p_price.select('[data-price]')[0].text.strip()